# 01 - Data Exploration

Initial look at the transformer monitoring data from SCADA systems and maintenance logs.

**Goal**: Understand data quality, distributions, and basic patterns before modelling.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 5)

%matplotlib inline

## 1. Load Raw Data

In [ ]:
sensor_df = pd.read_csv('data/raw/sensor_readings.csv', parse_dates=['timestamp'])
maint_df = pd.read_csv('data/raw/maintenance_log.csv', parse_dates=['install_date', 'last_inspection_date', 'event_date'])

print(f'Sensor readings: {sensor_df.shape}')
print(f'Maintenance log: {maint_df.shape}')

In [ ]:
sensor_df.head(10)

In [ ]:
sensor_df.info()

In [ ]:
maint_df.head(10)

## 2. Data Quality Check

In [ ]:
# Missing values
missing = sensor_df.isnull().sum()
missing_pct = (missing / len(sensor_df) * 100).round(2)

print('Missing values per column:')
print(pd.DataFrame({'count': missing, 'pct': missing_pct}))
print(f'\nTotal rows: {len(sensor_df)}')
print(f'Complete rows: {sensor_df.dropna().shape[0]}')
print(f'Missing rate: {1 - sensor_df.dropna().shape[0]/len(sensor_df):.2%}')

Interesting - the missing values are at the row level (entire rows missing, not individual columns). This is consistent with SCADA data where a sensor goes offline entirely rather than partially.

Let's check if missing data is random or concentrated on certain equipment.

In [ ]:
# Check missing data distribution across equipment
expected_per_equip = sensor_df['timestamp'].nunique()
actual_per_equip = sensor_df.groupby('equipment_id').size()
missing_per_equip = expected_per_equip - actual_per_equip

print(f'Expected readings per equipment: ~{expected_per_equip}')
print(f'\nMissing readings per equipment:')
print(missing_per_equip.describe())

Missing data looks roughly uniform across equipment. No single transformer has excessive missing readings. We can safely use forward-fill or just drop NaN rows.

## 3. Sensor Reading Distributions

In [ ]:
# Basic statistics
sensor_df.describe().round(2)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

columns = ['oil_temp_top_celsius', 'oil_temp_bottom_celsius', 'load_mva',
           'ambient_temp_celsius', 'winding_temp_celsius', 'cooling_fan_status']
titles = ['Oil Temp (Top)', 'Oil Temp (Bottom)', 'Load (MVA)',
          'Ambient Temp', 'Winding Temp', 'Cooling Fan Status']

for ax, col, title in zip(axes.flat, columns, titles):
    if col == 'cooling_fan_status':
        sensor_df[col].value_counts().plot(kind='bar', ax=ax, color=['steelblue', 'coral'])
    else:
        sensor_df[col].hist(bins=50, ax=ax, color='steelblue', alpha=0.7)
    ax.set_title(title, fontsize=12)
    ax.set_xlabel('')

plt.suptitle('Sensor Reading Distributions', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

Observations:
- Oil temperatures roughly normal, centred around 70-72°C (top) and 62-65°C (bottom)
- Load is bimodal - some transformers run consistently higher than others
- Ambient temperature shows clear seasonal pattern (bimodal due to summer/winter)
- Cooling fans are ON about 40-50% of the time
- Winding temp is the hottest reading, as expected (80-85°C typical)

## 4. Temporal Patterns

In [ ]:
# Pick a sample transformer to visualise
sample_id = 'TRANS_007'
sample = sensor_df[sensor_df['equipment_id'] == sample_id].sort_values('timestamp')

fig, axes = plt.subplots(4, 1, figsize=(16, 14), sharex=True)

axes[0].plot(sample['timestamp'], sample['oil_temp_top_celsius'], 'b-', alpha=0.6, linewidth=0.8)
axes[0].set_ylabel('Oil Temp Top (°C)')
axes[0].set_title(f'Sensor Readings Over Time - {sample_id}')

axes[1].plot(sample['timestamp'], sample['winding_temp_celsius'], 'r-', alpha=0.6, linewidth=0.8)
axes[1].set_ylabel('Winding Temp (°C)')

axes[2].plot(sample['timestamp'], sample['load_mva'], 'g-', alpha=0.6, linewidth=0.8)
axes[2].set_ylabel('Load (MVA)')

axes[3].plot(sample['timestamp'], sample['ambient_temp_celsius'], 'orange', alpha=0.6, linewidth=0.8)
axes[3].set_ylabel('Ambient Temp (°C)')
axes[3].set_xlabel('Date')

plt.tight_layout()
plt.show()

Clear seasonal patterns in ambient temperature driving oil temp and load. The summer peaks (Jul-Aug) and winter troughs are visible. Load also shows seasonal variation - higher in summer months likely due to increased cooling demand.

Let me check if TRANS_007 had any events...

In [ ]:
# Check maintenance events for this transformer
maint_df[maint_df['equipment_id'] == sample_id]

TRANS_007 had a failure event - that spike in temperatures before the failure date is interesting. We'll explore this more in notebook 02.

## 5. Correlation Analysis

In [ ]:
# Correlation heatmap for numeric sensor columns
numeric_cols = ['oil_temp_top_celsius', 'oil_temp_bottom_celsius', 'load_mva',
                'ambient_temp_celsius', 'cooling_fan_status', 'winding_temp_celsius']

corr = sensor_df[numeric_cols].corr()

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            square=True, linewidths=0.5, ax=ax)
ax.set_title('Sensor Correlation Matrix', fontsize=14)
plt.tight_layout()
plt.show()

Key correlations:
- **Oil top & winding temp**: Very high (~0.85+) - makes physical sense, winding heats the oil
- **Load & oil temps**: Moderate positive - higher load = more heat
- **Ambient & oil temps**: Weak-moderate positive - environmental effect
- **Cooling fan & oil temp**: Positive - fans kick in when temps rise

No surprising correlations. Physics checks out.

## 6. Equipment Metadata

In [ ]:
# Look at equipment age distribution
equipment_info = maint_df.groupby('equipment_id').agg(
    install_date=('install_date', 'first'),
    last_inspection=('last_inspection_date', 'max'),
    n_events=('event_type', 'count'),
    n_failures=('event_type', lambda x: (x == 'FAILURE').sum())
).reset_index()

equipment_info['age_years'] = (pd.Timestamp('2024-01-01') - equipment_info['install_date']).dt.days / 365.25

print(f'Equipment count: {len(equipment_info)}')
print(f'\nAge distribution:')
print(equipment_info['age_years'].describe().round(1))
print(f'\nFailure count distribution:')
print(equipment_info['n_failures'].value_counts().sort_index())

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(equipment_info['age_years'], bins=15, color='steelblue', alpha=0.7, edgecolor='white')
axes[0].set_xlabel('Equipment Age (years)')
axes[0].set_ylabel('Count')
axes[0].set_title('Equipment Age Distribution')

# Failures by event type
event_counts = maint_df['event_type'].value_counts()
event_counts.plot(kind='barh', ax=axes[1], color='steelblue', alpha=0.7)
axes[1].set_xlabel('Count')
axes[1].set_title('Maintenance Event Types')

plt.tight_layout()
plt.show()

## 7. Key Takeaways

1. **Data quality**: ~5% missing at row level, evenly distributed. Manageable.
2. **Seasonal patterns**: Strong seasonality in ambient temp, moderate in load and oil temps
3. **Correlations**: Oil temp, winding temp, and load are correlated (physically expected)
4. **Equipment age**: Ranges from ~9 to 29 years. Older fleet.
5. **Event distribution**: 11 failures across 2 years, plus scheduled maintenance, cooling repairs, oil changes
6. **Next step**: Investigate failure patterns in detail (notebook 02)